In [1]:
import pandas as pd
import data_disaggregation as dd # data_disaggregation>=0.8
import pint_pandas

In [2]:
# create dimensions as pandas Index
idx_month = pd.Index(["1", "2"], name="month")
idx_hour = pd.Index(["11", "21", "22"], name="hour")
idx_region = pd.Index(["a", "b"], name="region")

# create multi dimensions as pandas MultiIndex
idx_region_month = pd.MultiIndex.from_product([idx_region, idx_month])
idx_region_hour = pd.MultiIndex.from_product([idx_region, idx_hour])


In [3]:
# create data as pandas series over dimensions
# example: series over region
s_region = pd.Series({"a": 10, "b": 30}, index=idx_region)
# or just rename the index
s_region = pd.Series({"a": 10, "b": 30}, dtype="pint[meter]").rename_axis(["region"])
s_region

c:\Users\c.winger\nobackup\bin\python\lib\site-packages\pint_pandas\pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of int32. Converting magnitudes to float.
  warnings.warn(


region
a    10.0
b    30.0
dtype: pint[meter]

In [4]:
# create (weighted) map between dimensions (in the simple case, all weights = 1)
# as a series, with all dimensions in the index
s_month = pd.Series({"1": 2, "2": 3}, index=idx_month)

# replicate weight map to a second dimension
s_month = s_month * pd.Series(1, index=idx_region_month)
s_month

region  month
a       1        2
        2        3
b       1        2
        2        3
dtype: int64

In [5]:
from data_disaggregation.base import create_map, get_idx_out


idx = get_idx_out(s_month, s_region.index)

map = create_map(s_month, s_region.index)

map = create_map(s_month, s_region.index, idx_region_month)
print(map)

#print(dd.VT_NumericExt.disagg(s_region, s_month))

print(dd.VT_NumericExt.disagg(s_region, map))

print(dd.VT_Numeric.disagg(s_region, s_month))

print(dd.VT_Numeric.disagg(s_region, map))

region  (region, month)
a       (a, 1)             2
        (a, 2)             3
b       (b, 1)             2
        (b, 2)             3
dtype: int64
[('region', 'month')]


ValueError: Length of names must match number of levels in MultiIndex.

In [ ]:
# Use the appropriate Vartype class to (dis-) aggregate the data using the weight map
dd.VT_NumericExt.disagg(s_region, s_month, size_t=pd.Series(1, index=idx_region_month))
dd.VT_NumericExt.disagg(s_region, s_month)